# Setting up Chroma
For the numerical exercises we will use the Chroma software suite https://github.com/JeffersonLab/chroma. Chroma can be compiled with different libraries to run calculations both on CPUs and GPUs. We will use precompiled CPU only version that runs on google colab.
Run the next 2 cells to download and set the environment for the exercises.


In [1]:
%rm -r /home/software /home/tests /home/chroma_gcolab/ /home/run*
%mkdir /home/software
%cd /home/software
!gdown https://drive.google.com/uc?id=16woEAYY0VPJqufF2574Q1AM2NFh13qH1
!unzip chroma.zip
!rm chroma.zip
!chmod u+rwx /home/software/chroma/install/chroma/bin/*
%cd /home
!git clone https://github.com/henrymonge/chroma_gcolab.git
!mv chroma_gcolab/tests .
%cd /home/chroma_gcolab
!git pull
! pip install mpi4py
!pip install lsqfit
!pip install h5py

rm: cannot remove '/home/run*': No such file or directory
/home/software
Downloading...
From (original): https://drive.google.com/uc?id=16woEAYY0VPJqufF2574Q1AM2NFh13qH1
From (redirected): https://drive.google.com/uc?id=16woEAYY0VPJqufF2574Q1AM2NFh13qH1&confirm=t&uuid=33c05961-a8ca-4f00-abaa-909d7d9fc852
To: /home/software/chroma.zip
100% 175M/175M [00:01<00:00, 102MB/s]
Archive:  chroma.zip
   creating: chroma/
  inflating: __MACOSX/._chroma       
   creating: chroma/install/
  inflating: __MACOSX/chroma/._install  
  inflating: chroma/.DS_Store        
  inflating: __MACOSX/chroma/._.DS_Store  
  inflating: chroma/hmc.prec_wilson.ini.xml  
  inflating: __MACOSX/chroma/._hmc.prec_wilson.ini.xml  
   creating: chroma/install/qdpxx/
  inflating: __MACOSX/chroma/install/._qdpxx  
   creating: chroma/install/chroma/
  inflating: __MACOSX/chroma/install/._chroma  
   creating: chroma/install/qmp/
  inflating: __MACOSX/chroma/install/._qmp  
  inflating: chroma/install/README   
  inflatin

In [2]:
%cd /home/software/chroma
%env TOPDIR_HIP=/home/software/chroma
%env INSTALLROOT=${TOPDIR_HIP}/install
%env LD_LIBRARY_PATH=/home/software/chroma/install/chroma/lib:/home/software/chroma/install/qdpxx/lib:/home/software/chroma/install/qmp/lib

#Load python modules

%cd /home/chroma_gcolab/modules
import importlib
import xml_input
import defs
importlib.reload(defs)


/home/software/chroma
env: TOPDIR_HIP=/home/software/chroma
env: INSTALLROOT=${TOPDIR_HIP}/install
env: LD_LIBRARY_PATH=/home/software/chroma/install/chroma/lib:/home/software/chroma/install/qdpxx/lib:/home/software/chroma/install/qmp/lib
/home/chroma_gcolab/modules


<module 'defs' from '/home/chroma_gcolab/modules/defs.py'>

#Lattice QCD Numerical Calculations

Lattice QCD calculations are commonly performed in three stages:

1. Gauge field generation
2. Quark propagator computation/Constructing observables from quark propagators
4. Extracting physical quantities

Our first exercise will be to generate a gauge field configuration using Chroma.

#Stage 2: Computing quark propagators

A quark propagator can be obtained by inverting the Dirac operator $D$. To build $D$, we will need a gauge field configuration and a discretization scheme. For the gauge field, we use the ones generated using Monte Carlo importance sampling. The action used in the gauge field generate defines the gauge and sea quark properties. The discretization scheme used to represent the Dirac operator on the Lattice will determine the properties of the valence quarks and the observables we compute with these quarks. The process of simulating the physical process on a finite volume and a finite grid introduces errors, and therefore the choice of the discretization is relevant for determining systematics errors.

Standard methods for inverting matrices are not efficient for sparse matrices such as the Dirac operator. Instead, one can solve the following equation:

\begin{align}
D P = S
\end{align}

where $P=D^{-1}$.

Solving the above system of linear equations is a well-known problem and there allows to use existing techniques such as iterative methods. An iterative method we will require performing the matrix operation $D P$ multiple times. In principle, you implement this operation using an array for $D$ and $P$, nevertheless as $D$ is sparse it is more efficient to implement the operation of $D$ on $P$; This saves both memory and computing time.

Once the iterative method has been implemented, what we need to the computer is the gauge field and source.


## Multi-mass inverters

For a propagator, the system we solve is at every space-time point is

\begin{align}
D_x P_x = S_x
\end{align}

The application of $D_x$ is independent of $x$ and thus we can distribute this calculate and compute the $V=L^3 T$ sites in parallel. Now, the solution $P_x$ depends on the source $S_x$ we pass, but the $D_x$ does not. Moreover, for every solve we need to load the parameters necessary to apply $D$. If you do not have enough work to do at a given $x$, after every application the workers will need to wait until the data is passed. For the Lattice QCD, this is the case. A technique used to increase performance is the multi-right hand side MRS. In the MRS, we pass multiple sources and solve at once. In this way, only the for first source we need to wait for the $D$ data. This technique requires more memory but reduces the communications.









In [34]:
params={}
params['VOL']='4 4 4 32'
params['ACTION']='WILSON'
params['MASS']='-0.5'
params['INVERTER']='CG_INVERTER'
params['RES']='1.0e-8'
params['ITER']='1000'
outdir='/home/run_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFileProp(params,outdir) #update input file
inic=ini+'.txt'
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1
!echo "Input file is {ini}"
!cp $ini $inic
!grep -r "CHROMA: total time =" $stdout


mkdir: cannot create directory ‘/home/run_0’: File exists
Input file is /home/run_0/prop.ini.xml
CHROMA: total time = 4.434064 secs


In [15]:
#!/home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1

# Mass dependence
Run multiple calculations changing the quark mass and compare timings.

In [39]:
#Run 1
params={}
params['VOL']='4 4 4 32'
params['ACTION']='WILSON'
params['MASS']='-0.05'
params['INVERTER']='CG_INVERTER'
params['RES']='1.0e-8'
params['ITER']='1000'
outdir='/home/run_1'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFileProp(params,outdir) #update input file
inic=ini+'.txt'
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1
!echo "Input file is {ini}"
!cp $ini $inic
!grep -r "CHROMA: total time =" $stdout
!grep -r " iterations." $stdout

mkdir: cannot create directory ‘/home/run_1’: File exists
Input file is /home/run_1/prop.ini.xml
CHROMA: total time = 10.102474 secs
CG_SOLVER: 105 iterations. Rsd = 2.11787374433626e-08 Relative Rsd = 3.60778500629568e-08
CG_SOLVER: 105 iterations. Rsd = 2.14391146287904e-08 Relative Rsd = 3.65220578678012e-08
CG_SOLVER: 109 iterations. Rsd = 1.32056499641507e-08 Relative Rsd = 2.24957441709932e-08
CG_SOLVER: 109 iterations. Rsd = 1.31656013104341e-08 Relative Rsd = 2.24279249049943e-08
CG_SOLVER: 105 iterations. Rsd = 2.1018109402858e-08 Relative Rsd = 3.58104190175246e-08
CG_SOLVER: 106 iterations. Rsd = 2.07968963179198e-08 Relative Rsd = 3.54379173564793e-08
CG_SOLVER: 105 iterations. Rsd = 2.27466743550685e-08 Relative Rsd = 3.87555285919018e-08
CG_SOLVER: 109 iterations. Rsd = 1.3927252384752e-08 Relative Rsd = 2.37320421022822e-08
CG_SOLVER: 105 iterations. Rsd = 2.2588945330058e-08 Relative Rsd = 3.84907917244316e-08
CG_SOLVER: 104 iterations. Rsd = 2.2514864752809e-08 Relativ

In [41]:
#Run 3 UNPREC_WILSON
params={}
params['VOL']='4 4 4 32'
params['ACTION']='UNPRECONDITIONED_WILSON'
params['MASS']='-0.05'
params['INVERTER']='CG_INVERTER'
params['RES']='1.0e-8'
params['ITER']='1000'
outdir='/home/run_1'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFileProp(params,outdir) #update input file
inic=ini+'.txt'
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1
!echo "Input file is {ini}"
!cp $ini $inic
!grep -r "CHROMA: total time =" $stdout
!grep -r " iterations." $stdout


mkdir: cannot create directory ‘/home/run_1’: File exists
Input file is /home/run_1/prop.ini.xml
CHROMA: total time = 25.089024 secs
CG_SOLVER: 265 iterations. Rsd = 6.42598662582617e-08 Relative Rsd = 6.42598662582617e-08
CG_SOLVER: 266 iterations. Rsd = 6.32549510313872e-08 Relative Rsd = 6.32549510313872e-08
CG_SOLVER: 266 iterations. Rsd = 6.63284467501006e-08 Relative Rsd = 6.63284467501006e-08
CG_SOLVER: 265 iterations. Rsd = 6.79246416738039e-08 Relative Rsd = 6.79246416738039e-08
CG_SOLVER: 277 iterations. Rsd = 4.50183047886464e-08 Relative Rsd = 4.50183047886464e-08
CG_SOLVER: 277 iterations. Rsd = 4.79997398751332e-08 Relative Rsd = 4.79997398751332e-08
CG_SOLVER: 277 iterations. Rsd = 4.67185662669531e-08 Relative Rsd = 4.67185662669531e-08
CG_SOLVER: 277 iterations. Rsd = 4.83775783090389e-08 Relative Rsd = 4.83775783090389e-08
CG_SOLVER: 277 iterations. Rsd = 4.78007973752266e-08 Relative Rsd = 4.78007973752265e-08
CG_SOLVER: 277 iterations. Rsd = 4.46728685950262e-08 Rel


## Computing the spectrum

Comment on contractions




# Extracting the mass spectrum



\begin{align}
C(t)=\sum_n e^{-E_nt}
\end{align}


\begin{align}
C(t)=\sum_n (e^{-E_nt} - e^{-E_n(T-t)})
\end{align}

# Hadron spectrum

Computing the hadron spectrum is fairly simple, at any given space-time point, what we need to do is to contract a set of matrices and then we perform a fourier transform.

The number of terms to contract depends of the observable and their symmetries. For computing the correlation function for a single particle the only thing we want to do is to do the contraction in parallel at every space-time point. When we compute more particles, there may be opportunities for reusing some data.


The next measurement we will perform is generating the hadron spectrum

 LatticePropagator anti_quark_prop =  Gamma(G5) * quark_prop_2 * Gamma(G5);

corr_fn = trace(adj(anti_quark_prop) * (Gamma(gamma_value) *
                    quark_prop_1 * Gamma(gamma_value)));







#Running Chroma

To maximize resource usage,we can split the problem.
In chroma, you can use -geom 1 1 1 1



In [ ]:
#params.update({'VOL':' 4 4 4 8','CFG':'WEAK_FIELD'})
params={}
outdir='/home/run_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFileHadspec(params,outdir) #update input file
!echo "Input file is {ini}"

mkdir: cannot create directory ‘/home/run_0’: File exists
Input file is /home/run_0/prop.ini.xml


In [ ]:
#!/home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1

# Extracting the mass spectrum

In [5]:
%cd /home/chroma_gcolab/corr_fitter/

import importlib
import io_params
importlib.reload(io_params)
from io_params import *
import numpy as np
import gvar as gv
import h5py as h5
from fit_functions import *
import lsqfit

plt.ion()

f=h5.File('../a09m260_a_avg_srcs0-7.h5')
particle='proton'  #kplus,lambda_z,omega_m,piplus,proton
isMeson=particle in ['pion','kminus','kplus','piplus']

path='gf1p0_w3p5_n45_M51p1_L58_a1p5/spec/ml0p00702_ms0p0481'
#Retrieve data
if isMeson:
    corr=f[path+f'/{particle}/psq_0/corr'][()].real
else:
    corr_spin_dn=f[path+f'/{particle}/psq_0/spin_dn'][()].real
    corr_spin_up=f[path+f'/{particle}/psq_0/spin_up'][()].real
    corr=(corr_spin_dn+corr_spin_up)  #spin average
corr=corr[:,:,1,0]

#time extent
T=corr.shape[1]

/home/chroma_gcolab/corr_fitter


In [7]:
#Pack the mean and std into a gvar
j=1
x=np.arange(T)
y=gv.dataset.avg_data(corr)
y_pj = np.roll(y,-j)
y_mj = np.roll(y,j)

#Compute the meff
if isMeson:
    meff =  1. / j * np.arccosh((y_pj+y_mj) / 2 / y )
else:
    meff =  1. / j * np.log( y / y_pj )

#Plot correlator
fig = plt.figure(1,figsize=(20,10))
ax=plt.subplot()
ax.errorbar(np.arange(T),gv.mean(y), yerr=gv.sdev(y),ms=ms,mew=mew,
                mfc='None',capsize=capsize,linestyle='None',alpha=0.9)
ax.set_xlabel('t')
ax.set_ylabel('C(t)')
plt.title(particle)
plt.show(block=False)

/tmp/ipython-input-7-3669048730.py:12: RuntimeWarning: invalid value encountered in log
  meff =  1. / j * np.log( y / y_pj )


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2000x1000 with 1 Axes>

In [6]:
#Pack the mean and std into a gvar
j=1
x=np.arange(T)
y=gv.dataset.avg_data(corr)
y_pj = np.roll(y,-j)
y_mj = np.roll(y,j)

#Compute the meff
if isMeson:
    meff =  1. / j * np.arccosh((y_pj+y_mj) / 2 / y )
else:
    meff =  1. / j * np.log( y / y_pj )

#Plot correlator
fig = plt.figure(1,figsize=(20,10))
ax=plt.subplot()
ax.errorbar(np.arange(T),gv.mean(y), yerr=gv.sdev(y),ms=ms,mew=mew,
                mfc='None',capsize=capsize,linestyle='None',alpha=0.9)
ax.set_xlabel('$t$')
ax.set_ylabel('$C(t)$')
plt.title(particle)
plt.show(block=False)

#Plot meff
fig = plt.figure(2,figsize=(20,10))
ax=plt.subplot()
ax.errorbar(np.arange(T),gv.mean(meff), yerr=gv.sdev(meff),ms=ms,mew=mew,
                mfc='None',capsize=capsize,linestyle='None',alpha=0.9)
ax.set_xlabel('$t$')
ax.set_ylabel('$m_{eff}$')
plt.title(particle)
plt.show()

p0 = None    # make larger fits go faster (opt.)
nstates=1
for nexp in range(1, nstates+1):
	print('************************************* nexp =', nexp)
	prior = make_prior(nexp)
	fit = lsqfit.nonlinear_fit(data=(x, y), fcn=fcn, prior=prior,p0=p0)
	print(fit)		  # print the fit results

	if nexp > 2:
		E = fit.p['E']	  # best-fit parameters
		a = fit.p['a0']
		print('E1/E0 =', E[1] / E[0], '  E2/E0 =', E[2] / E[0])
		print('a1/a0 =', a[1] / a[0], '  a2/a0 =', a[2] / a[0])
		if fit.chi2 / fit.dof < 1.:
			p0 = fit.pmean	  # starting point for next fit (opt.)
		print()

		# error budget analysis
		outputs = {	'E1/E0':E[1]/E[0], 'E2/E0':E[2]/E[0],
    	            'a1/a0':a[1]/a[0], 'a2/a0':a[2]/a[0]}

		inputs = {'E':fit.prior['E'], 'a0':fit.prior['a0'],'a1':fit.prior['a1'], 'y':y}
		print('================= Error Budget Analysis')
		print(fit.fmt_values(outputs))
		print(fit.fmt_errorbudget(outputs,inputs))

#posterior fit.p['E']
#meff_fit=fit.p['E'][0]
#ax.fill_between(x,meff_fit.mean-meff_fit.sdev, meff_fit.mean+meff_fit.sdev,facecolor='k',alpha=0.25)

plt.ioff()


/tmp/ipython-input-6-2751305547.py:12: RuntimeWarning: invalid value encountered in log
  meff =  1. / j * np.log( y / y_pj )


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2000x1000 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2000x1000 with 1 Axes>

************************************* nexp = 1
Least Squares Fit:
  chi2/dof [dof] = 1.7e+04 [96]    Q = 0    logGBF = -8.1323e+05

Parameters:
           A0 0   -2.8647(30)e-07       [  0.50 (40) ]  *
           A1 0         0.50 (40)       [  0.50 (40) ]  
            E 0       0.7354 (78)       [  1.00 (40) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 22/0.6s)
  fitter = scipy_least_squares    method = trf



#Computational cost volume dependence

The computational cost of HMC is $\propto V^{\frac54}$.

Perform 4 runs with different volumes. Does the run time increases as $\propto V^{\frac54}$?

In [ ]:
#Run 1
params['VOL']= ' 4 4 4 8'#,' 4 4 4 16',' 4 4 4 32',' 8 8 8 16'
outdir='/home/run_VOL_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFile(params,ini,outdir) #update input file
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1

!echo 'The stdout file is $stdout' # Look for HMC: total time in this files
!grep -r "HMC: total time" $stdout #

mkdir: cannot create directory ‘/home/run_VOL_0’: File exists
/home/run_VOL_0/hmc.ini.xml
The stdout file is $stdout
HMC: total time = 28.09331 secs


# Acceptance rate

Every molecular dynamics trajectory generates a new gauge field configuration. This new configuration will be accepted depending on the acceptance probability.

Perform 4 calculations changing the trajectory length and the number of steps. How does the acceptance rate changes?

In [ ]:
#Run 1
params['INTGRTOR_STEPS']= '64'
params['INTGRTOR_TAU']= '1'
outdir='/home/run_ACP_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFile(params,ini,outdir) #update input file
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1

!echo 'The stdout file is $stdout' # Look for AccProb =  in this files
!grep -r "Delta H = " $stdout
!grep -r "AccProb = " $stdout
!grep -r "AcceptP = " $stdout

mkdir: cannot create directory ‘/home/run_ACP_0’: File exists
The stdout file is $stdout
Delta H = 1.03117174726685
AccProb = 0.356588883626945
AcceptP = 1


In [ ]:
#Run 2
#To fill in copy above and change VOL and outdir number

In [ ]:
#Run 3
#To fill in copy above and change VOL and outdir number

In [ ]:
#Run 4
#To fill in copy above and change VOL and outdir number

# Preconditioning

Every molecular dynamics trajectory generates a new gauge field configuration. This new configuration will be accepted depending on the acceptance probability.

Perform 4 calculations changing the trajectory length and the number of steps. How does the acceptance rate changes?

In [ ]:
#Run 1
params['INTGRTOR_STEPS']= '64'
params['INTGRTOR_TAU']= '1'
outdir='/home/run_ACP_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFile(params,ini,outdir) #update input file
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/chroma -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1

!echo 'The stdout file is $stdout' # Look for AccProb =  in this files
!grep -r "Delta H = " $stdout
!grep -r "AccProb = " $stdout
!grep -r "AcceptP = " $stdout

mkdir: cannot create directory ‘/home/run_ACP_0’: File exists
The stdout file is $stdout
Delta H = 1.03117174726685
AccProb = 0.356588883626945
AcceptP = 1


In [ ]:
params['VOL']=' 16 16 16 48'#' 4 4 4 8',' 4 4 4 16',' 4 4 4 32',' 8 8 8 16'
params['Monomials']['TWO_FLAVOR_EOPREC']['CLOV_COEFF']='3.5'
params['WARMUPS']='1'
params['PROD_UPDATES']='1'
params['NUPDATES']='1'
mass='-0.5' #-5,-0.5,5.0
results={}
results['MASS']={}
params['Monomials']['TWO_FLAVOR_EOPREC']['MASS']=mass
defs.writeInputFile(params,ini) #update input file
! mpiexec --allow-run-as-root --oversubscribe -np 2 /home/software/chroma/install/chroma/bin/hmc -geom 1 1 1 2 -i $ini -o $out -l $log > $stdout 2>&1
temp=!grep -r "HMC: total time" $stdout
ttime=float(temp[0].split('= ')[1].split(' ')[0])
if mass not in results['MASS']:
  results['MASS'][mass]={}
results['MASS'][mass]['time']=ttime

iters=!grep -r "iterations."  $stdout
results['MASS'][mass]['iters']=iters[0].split(' iterations')[0].split(' ')[-1]

for m in results['MASS']:
  print(m,results['MASS'][m]['time'])



In [ ]:
print('Run results')
!grep -r "Delta H = " $stdout
!grep -r "AccProb = " $stdout
!grep -r "AcceptP = " $stdout
!grep -r "HMC: total time" $stdout
#!grep -r "CG_SOLVER: " $stdout | grep iterations
'''
Run results
Delta H = 1.03117174726685
AccProb = 0.356588883626945
AcceptP = 1
HMC: total time = 34.127923 secs
'''

Run results
Delta H = 1.03117174727413
AccProb = 0.35658888362435
AcceptP = 1
HMC: total time = 38.53905 secs


'\nRun results\nDelta H = 1.03117174726685\nAccProb = 0.356588883626945\nAcceptP = 1\nHMC: total time = 34.127923 secs\n'

In [ ]:
for m in results['MASS']:
  print(m,results['MASS'][m]['iters'])

-0.005 26
-0.5 26


# Quark Mass dependence

Now we will performed multiple calculations to explore the quark mass dependence

#Exploring more calculations

Below are the cells with all the parameters that you can modify to perform new simulations.

Changes to try:
1.   MD trajectory length: TAU
2.   Fermion masses: MASS
3.   Gauge coupling: BETA
4.   MD steps: NSTEPS
5.   Gauge field type

In [ ]:
#Monomials
#Gauge Monomials
#Fermion Monomials
params={}
params['Monomials']={}

params['Monomials']['GAUGE_MONOMIAL']={'ACTION':'WILSON_GAUGEACT',
                                      'BETA':'3.4','ID':'HMC::gauge'}
params['Monomials']['ONE_FLAVOR_EOPREC']={'ACTION':'CLOVER',
                                          'MASS':'-0.04','CLOV_COEFF':'1.2','ID':'HMC::strange'}
params['Monomials']['TWO_FLAVOR_EOPREC']={'ACTION':'CLOVER','MASS':'-0.02',
                                          'CLOV_COEFF':'1.2','ID':'HMC::light'}

#MD integrator
#Choose the integrator parameters
params['INTGRTOR']='LCM_STS_LEAPFROG'
params['INTGRTOR_STEPS']='64'
params['INTGRTOR_TAU']='1'

#HMC updates
params['WARMUPS']='1'
params['PROD_UPDATES']='1'
params['NUPDATES']='1'

#Lattice and gauge field
params['VOL']=' 4 4 4 8'
params['CFG']='WEAK_FIELD'  #DISORDERED,UNIT

defs.setMonomialIds(params)
defs.display_action(params['Monomials'])



**************************************************
               My chosen action is:
**************************************************



<IPython.core.display.Latex object>

In [ ]:
outdir='/home/run_0'
%mkdir $outdir
ini,out,log,stdout=defs.writeInputFile(params,ini,outdir) #update input file
!echo "Input file is {ini}"
! mpiexec --allow-run-as-root --oversubscribe -np 1 /home/software/chroma/install/chroma/bin/hmc -geom 1 1 1 1 -i $ini -o $out -l $log > $stdout 2>&1


mkdir: cannot create directory ‘/home/run_0’: File exists
Input file is /home/run_0/hmc.ini.xml


In [ ]:
print('Run results')
!grep -r "Delta H = " $stdout
!grep -r "AccProb = " $stdout
!grep -r "AcceptP = " $stdout
!grep -r "HMC: total time" $stdout
#!grep -r "CG_SOLVER: " $stdout | grep iterations

Run results
Delta H = -5.09948819451665
AccProb = 1
AcceptP = 1
HMC: total time = 5.67467 secs


# Questions

##Questions to answer



1.   How does acceptance probability changes with different parameters?
2.   How does time changes with changes in the parameters?


